In [ ]:
import pandas as pd
import re, logging, os, sys, json

In [ ]:
def _extract_generic_data(self, main_key: str, data: list, pattern: str):
    final_dict = {}
    for text in data:
        text = re.sub(self.REGEX['escape'], "", text.strip())
        matches = re.findall(self.REGEX[pattern], text, re.IGNORECASE)
        for key, value in matches:
            final_dict[key] = value
    return {main_key: final_dict}

def _extract_str_data(self, key: str, data: list):
    return {key: ' '.join(data)}

def _extract_manager_data(self, main_key: str, data: list, pattern:str):
    final_list = []
    for i in range(0, len(data), 3):
        txt = " ".join(data[i:i+3])
        txt = re.sub(self.REGEX['escape'], "", txt).strip()
        if matches := re.findall(self.REGEX[pattern], txt, re.IGNORECASE):
            for match in matches:
                name, desig, since, exp = match
                final_list.append({
                    "name": name,
                    "designation": desig,
                    "managing_since": since,
                    "experience": exp
                })
    return {main_key: final_list}

def _extract_dum_data(self, key, data):
    return {key: data}

def _extract_scheme_data(self,main_key:str,data:list, pattern:str):
    regex_ = self.REGEX[pattern] #list
    mention_start = regex_[:-1]
    mention_end = regex_[1:]

    patterns = [r"({start})\s*(.+?)\s*({end}|$)".format(start=start, end=end)
        for start, end in zip(mention_start, mention_end)]
    
    final_dict = {}
    scheme_data = " ".join(data)
    for pattern in patterns:
        if matches:= re.findall(pattern, scheme_data, re.DOTALL|re.IGNORECASE):
            for match in matches:
                key, value, dummy = match
                value = value.strip()
                final_dict[key] = value
    
    return {main_key:final_dict}

def match_regex_to_content(self, string: str, data: list, *args):
    for pattern, (func_name, regex_key) in self.PATTERN_TO_FUNCTION.items():
        if re.match(pattern, string, re.IGNORECASE):
            func = getattr(self, func_name)
            if regex_key:
                return func(string, data, regex_key)
            return func(string, data)
    return self._extract_dum_data(string, data)

def _extract_load_data(self,main_key:str,data:list, pattern:str):
        load_data = " ".join(data)
        load_data = re.sub(r"[\^*\$:;~]","",load_data).strip()
        final_dict = {}
        if match:= re.findall(self.REGEX[pattern],load_data.strip(), re.IGNORECASE):
            exit_,entry_ = match[0]
            final_dict['entry_load'] = entry_,
            final_dict['exit_load'] = exit_
        return {main_key:final_dict}

In [ ]:
r'([\d,]+).*?([\d,]+)'

In [ ]:
pattern = r'(?:Entry Load)?\s*(Nil|.*?)\s*Exit Load\s*(.*)$'
for fund, content in final_text.items():
    check = 'scheme_details.load_structure'
    if check in content:
        text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "",content[check]).strip()
        print(text)
        match = re.findall(pattern,text, re.IGNORECASE)
        print(match)
        
pattern = r'(AUM|Monthly Average AUM)\s*([\d,.]+)'
for fund, content in final_text.items():
    check = 'aum'
    if check in content:
        for text in  content[check]:
            text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "",text).strip()
            print(text)
            match = re.findall(pattern,text, re.IGNORECASE)
            print(match)
            
pattern = r'(?:Entry Load)?\s*(Nil|.*?)\s*Exit Load\s*(.*)$'
for fund, content in final_text.items():
    check = 'scheme_details.load_structure'
    if check in content:
        all_text = " ".join(content[check])
        all_text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "", all_text).strip()
        print(all_text)
        print(re.findall(pattern,all_text, re.IGNORECASE))

In [ ]:
min_amt = "([\\d,]+).*?([\\d,]+)"

In [ ]:
mention_start = [
        "Date of Allotment",
        "Fund Manager",
        "Fund Size",
        "Load Structure",
        "Benchmark",
        "Minimum Additional",
        r"Minimum Redemption\s*\/\s*Switch-out Amount",
    ]
    
mention_end = mention_start[1:] + ["End_of_Data"]

# Generate regex patterns dynamically
patterns = [r"({start}\s*)(.+?)({end}|$)".format(start=start, end=end)
    for start, end in zip(mention_start, mention_end)]
final_dict = {}

In [ ]:
def get_unique_keys(json_files):
    unique_keys = set()  # Store unique keys

    for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)

                for fund_name, fund_data in data.items():
                    if isinstance(fund_data, dict):
                        extract_keys(fund_data, unique_keys)

        except Exception as e:
            print(f"Error reading {file}: {e}")

    return unique_keys

def extract_keys(data, key_set):
    """Recursively extract keys from JSON data."""
    if isinstance(data, dict):
        for key, value in data.items():
            key_set.add("_".join(key.lower().split(" ")))
            extract_keys(value, key_set)
    elif isinstance(data, list):
        for item in data:
            extract_keys(item, key_set)

In [ ]:
json_folder = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output"  # Update this to your folder path
json_files = [os.path.join(json_folder, file) for file in os.listdir(json_folder) if file.endswith('.json')]
unique_keys = get_unique_keys(json_files)


In [ ]:
import re, os, pprint
from app.pdfParse import Reader
from app.fundRegex import FundRegex
import fitz

class FranklinTempleton(Reader):
    PARAMS = {
        'fund': [[25,20],r"^(Franklin|Templeton).*$",[16,24],[-65794]], #[flag], regex_fund_name, range(font_size), [font_color]
        'clip_bbox': [(0,100,180,812)],
        'line_x': 180.0,
        'data': [[6,9],[-16751720],30.0,['ZurichBT-BoldCondensed']], #sizes, color, set_size font_name
        'content_size':[30.0,10.0]
    }
    
    REGEX = {
        'nav': r'(Growth Plan|IDCW Plan|Direct Growth Plan|Direct IDCW Plan)\s*([\d\-,]+\.\d+)',
        'metric': r'(Sharpe Ratio|Standard Deviation|Beta|Annualised)[\s]+([\d\-,]+\.\d+)',
        'expense': r'(Regular|Direct)\s*([\d,.-]+)',
        'load': r'ENTRY LOA(.*?)\s*EXIT LOAD\s*(.*?)$',
        'aum': r'(Month End|Monthly Average)\s*([\d,.\-]+)',
        'ptr':r'^(Portfolio Turnover|Total Portfolio).*\s*([\d\-,]+\.\d+)',
        # 'manager': r'([A-Za-z\s]+)\s*\(Managing Since\s*([A-Za-z0-9\s]+) and overall experience of ([a-z0-9\s]+)\)',
        'escape': r'[^A-Za-z0-9\s\-\(\).,]+'
    }

    PATTERN_TO_FUNCTION = {
        r"^(date|benchmark|investment|type_of|scheme|multiples|minimum).*":("_extract_str_data", None),
        r"^nav.*":("_extract_generic_data", "nav"),
        # r"^fund_mana.*": self.__extract_manager_data,
        r"^aum.*": ("_extract_generic_data", 'aum'),
        r"^portfolio_turnover_ratio": ("_extract_generic_data", 'ptr'),
        r"^load": ("_extract_load_data", 'load'),
        r"^metrics": ("_extract_generic_data", "metric"),
    }

    def __init__(self, paths_config: str):
        super().__init__(paths_config, self.PARAMS)
    
    def _extract_str_data(self, key: str, data: list):
        return {key: ' '.join(data)}
    
    def _extract_dummy_data(self, main_key:str, data:list):
        return{main_key:data}
    
    def _extract_generic_data(self, main_key: str, data: list, pattern: str):
        final_dict = {}
        for text in data:
            text = re.sub(self.REGEX['escape'], "", text.strip())
            matches = re.findall(self.REGEX[pattern], text, re.IGNORECASE)
            for match in matches:
                if len(match) == 2:
                    key, value = match
                    final_dict[key.strip()] = value.strip()
                elif len(match) == 3:
                    key,v1,v2 = match
                    final_dict[key.strip()] = v1.strip()
        return {main_key: final_dict}
    
    def _extract_manager_data(self, main_key: str, data: list, pattern:str):
        manager_data = " ".join(data)
        manager_data = re.sub(self.REGEX[pattern], "", manager_data).strip()
        final_list = []
        if matches := re.findall(self.REGEX['manager'], manager_data, re.IGNORECASE):
            for match in matches:
                name, since, exp = match
                final_list.append({
                    "name": name.strip(),
                    "designation": "",
                    "managing_since": since.strip(),
                    "experience": exp.strip()
                })
        return {main_key: final_list}
    
    def _extract_load_data(self,main_key:str,data:list, pattern:str):
        load_data = " ".join(data)
        load_data = re.sub(self.REGEX['escape'], "", load_data).strip()
        final_dict = {}
        if match:= re.findall(self.REGEX[pattern],load_data.strip(), re.IGNORECASE):
            exit_,entry_ = match[0]
            final_dict['entry_load'] = entry_,
            final_dict['exit_load'] = exit_
        return {main_key:final_dict}
        
    
    def match_regex_to_content(self, string: str, data: list, *args):
        for pattern, (func_name, regex_key) in self.PATTERN_TO_FUNCTION.items():
            if re.match(pattern, string, re.IGNORECASE):
                func = getattr(self, func_name)
                if regex_key and regex_key in self.REGEX:
                    return func(string, data, regex_key)
                return func(string, data)
        return self._extract_dummy_data(string, data)


In [ ]:
def merge_data(data, keys, target_key):
    """
    Merges values of two keys inside each fund and stores the result in target_key.

    :param data: Dictionary containing funds.
    :param keys: List of two keys whose values need to be merged.
    :param target_key: The key where the merged result should be stored.
    """
    if len(keys) != 2:
        raise ValueError("Exactly two keys must be provided for merging.")

    key1, key2 = keys

    for fund, values in data.items():
        if key1 in values and key2 in values:
            val1, val2 = values[key1], values[key2]

            if isinstance(val1, str) and isinstance(val2, str):
                values[target_key] = val1 + val2
            elif isinstance(val1, list) and isinstance(val2, list):
                values[target_key] = val1 + val2
            elif isinstance(val1, dict) and isinstance(val2, dict):
                values[target_key] = {**val1, **val2}
            else:
                raise TypeError(f"Cannot merge different data types: {type(val1)} and {type(val2)}")
    
    return data

# Example Usage
sample = {
    "fund_name1": {
        "a": "Hello",
        "b": " World",
        "c": {},
    },
    "fund_name2": {
        "a": [1, 2, 3],
        "b": [4, 5, 6],
        "c": {},
    },
}

merged_sample = merge_data(sample, ["a", "b"], "a")
print(merged_sample)


IMPORTANT CODE DONT DELETE

In [ ]:
"""wip"""
class DSP(Reader):
    
    PARAMS = {
        'fund': [[20,16], r'^(DSP|Bharat).*(Fund|ETF|FTF|FOF)$|^(DSP|Bharat)',[14,24],[-1]],
        'clip_bbox': [(0,5,120,812),],#[480,5,596,812]],
        'line_x': 120.0,
        'data': [[7,10], [-16777216], 30.0, ['TrebuchetMS-Bold']],
        'content_size':[30.0,10.0]
    }
    
    
    def __init__(self, paths_config:str):
        super().__init__(paths_config, self.PARAMS)
        
    #REGEX
    def __return_all_data(self,main_key,data:list):
        return {main_key:data}
    
    def __extract_inv_data(self,main_key:str, data:list):            
        return {main_key: ' '.join(data)}
    
    #MAPPING
    def match_regex_to_content(self, string:str, data:list):
        check_header = string
        if re.match(r"^investment.*", check_header, re.IGNORECASE):
            return self.__extract_inv_data(string,data)
        # elif re.match(r"^aum.*", check_header, re.IGNORECASE):
        #     return self.__extract_aum_data(string, data)
        # elif re.match(r"^fund_mana.*", check_header, re.IGNORECASE):
        #     return self.__extract_manager_data(string, data)
        # elif re.match(r"^metrics.*", check_header, re.IGNORECASE):
        #     return self.__extract_metric_data(string, data)
        return self.__return_all_data(string,data) 

""" thissssssssssssssssssssssss"""
class HSBC(Reader):
    
    PARAMS = {
        'fund': [[20,16], r'^(HSBC|Bharat).*Fund$',[12,20],[-1237724]],
        'clip_bbox': [(0,100,180,812)],
        'line_x': 180.0,
        'data': [[6,8], [-16777216], 30.0, ['Arial-BoldMT']],
        'content_size':[30.0,10.0]
    }
    
    def __init__(self,paths_config:str):
        super().__init__(paths_config, self.PARAMS)
        
    #Fund REGEX
    def __return_all_data(self,key,data:list):
        return {key:data}
    
    def __extract_inv_data(self,key:str, data:list):            
        return {key: ' '.join(data)}
    
    #MAPPING
    def match_regex_to_content(self, string:str, data:list):
        check_header = string
        if re.match(r"^(investment).*", check_header, re.IGNORECASE):
            return self.__extract_inv_data(string,data)
        # elif re.match(r"^total.*", check_header, re.IGNORECASE):
        #     return self.__extract_total_expense_data(string, data)
        # elif re.match(r"^nav.*", check_header, re.IGNORECASE):
        #     return self.__extract_nav_data(string, data)
        # elif re.match(r"^portfolio.*", check_header, re.IGNORECASE): #error here portfolio is read
        #     return self.__extract_scheme_data(string, data)
        return self.__return_all_data(string,data)     

class LIC(Reader):
    
    PARAMS = {
        'fund': [[20,16], r'^(LIC|Bharat).*(Fund|ETF|FTF|FOF)$',[12,20],[-15319437]],
        'clip_bbox': [(0,5,150,812),],
        'line_x': 150.0,
        'data': [[5,8], [-15445130,-14590595], 30.0, ['Frutiger-Bold']],
        "content_size":[30.0,10.0]
    }
    def __init__(self,paths_config:str):
        super().__init__(paths_config, self.PARAMS)          
                    
class QuantMF(Reader):
    PARAMS = {
        'fund': [[16,0], r'^(quant).*(Fund|ETF|EOF|FOF|FTF|Path)$',[16,24],[-13604430]],
        'clip_bbox': [(0,5,180,812)],
        'line_x': 180.0,
        'data': [[6,11], [-16777216], 30.0, ['Calibri,Bold',]],
        'content_size':[30.0,10.0]
        }
    
    def __init__(self, paths_config:str):
        super().__init__(paths_config,self.PARAMS)

class PPFAIS(Reader):
    PARAMS = {
        'fund': [[16,20], r'^(Parag).*',[14,24],[-16777216,-13159371,-14869475]],
        'clip_bbox': [(0,65,290,812)],
        'line_x': 290.0,
        'data': [[6,10], [-65794], 30.0, ['Arial-BoldMT',]]
        }
    
    def __init__(self, paths_config:str):
        super().__init__(paths_config,self.PARAMS)
        
    #REGEX
    def __return_all_data(self,main_key,data:list):
        return {main_key:data}
    
    def __extract_inv_data(self,main_key:str, data:list):            
        return {main_key: ' '.join(data)}
    
    #MAPPING
    def match_regex_to_content(self, string: str, data: list):
        pattern_to_function = {
            r"^investment.*": self.__extract_inv_data,
            # r"^scheme.*": self.__extract_scheme_data,
            # r"^metrics.*": self.__extract_metric_data,
        }

        for pattern, func in pattern_to_function.items():
            if re.match(pattern, string, re.IGNORECASE):
                return func(string, data)

        return self.__return_all_data(string, data)
    
class SBI(Reader):
    
    PARAMS = {
        'fund': [[16,0], r'^().*',[12,20],[-12371562]],
        'clip_bbox': [(0,5,180,812)],
        'line_x': 180.0,
        'data': [[6,11], [-12371562], 30.0, ['Calibri,Bold',]]}
    
    def __init__(self, paths_config:str):
        super().__init__(paths_config,self.PARAMS)
          
class PGIM(Reader):
    PARAMS = {
        'fund': [[20,4],r'^.*(Plan|Sensex|Fund|Path|ETF|FOF|EOF|Funds)$',[16,30],[-1]], #FUND NAME DETAILS order-> flag, regex_fund_name, font_size, font_color
        'clip_bbox': [(0,115,210,812)],
        'line_x': 210.0,
        'data': [[6,12],[-1],30.0,['PrudentialModern-Bold']], #sizes, color, set_size
        'content_size':[30.0,10.0]
    }
    
    def __init__(self, path: str,dry:str,fin:str, rep:str):
        super().__init__(path,dry,fin,rep, self.PARAMS)
    
    #Fund Regex  
    def __return_all_data(self,main_key:str,data:list):
        return{main_key:data}
    def __extract_fund_data(self,main_key:str, data:list):
        return {main_key:data}
    def __extract_invest_data(self,main_key:str,data:list):
        return {main_key: " ".join(data)}
    
    #MAPPING FUNCTION
    def match_regex_to_content(self, string: str, data: list):
        pattern_to_function = {
            r"^(investment|minimum|entry|exit|load|plans|scheme_launch|benchmark).*": self.__extract_invest_data,
            r"^fund_mana.*": self.__extract_fund_data,
        }

        for pattern, func in pattern_to_function.items():
            if re.match(pattern, string, re.IGNORECASE):
                return func(string, data)

        return self.__return_all_data(string, data)


#something


In [ ]:
#first
final_dic = {
    "metadata": [],
    "records": []
}

#second
meta_data = {
    "document_name": "",
    "file_type": "",
    "process_date": ""
}
record = {
            "amc_name": "",
            "benchmark_index":[],
            "field_location": [],
            "fund_manager": [],
            "load": [],
            "main_scheme_name": "",
            "metrics": [],
            "min_addl_amt": "",
            "min_addl_multiple":"",
            "min_amt":"",
            "monthly_aaum_date":"",
            "monthly_aaum_value":"",
            "mutual_fund_name": "",
            "scheme_launch_date":""
            
        }

#third  
fund_manager = {
    "managing_fund_since":"",
    "name": "",
    "qualification": "",
    "total_exp": ""
}
load = {
    "type": "",
    "comment": ""
}   
metrics= {
    "name":"",
    "value":""
}    
field_location = {
    "amc_name": "",
    "benchmark_index": "",
    "count": 0,
    "fund_manager_managing_fund_since": "",
    "fund_manager_name": "",
    "fund_manager_total_exp": "",
    "load_entry": "",
    "load_exit": "",
    "main_scheme_name": "",
    "metrics_beta": "",
    "metrics_port_turnover_ratio": "",
    "metrics_r_squared_ratio": "",
    "metrics_sharpe": "",
    "metrics_std_dev": "",
    "metrics_treynor_ratio": "",
    "min_addl_amt": "",
    "min_addl_amt_multiple": "",
    "min_amt": "",
    "min_amt_multiple": "",
    "monthly_aaum_date": "",
    "monthly_aaum_value": "",
    "mutual_fund_name": "",
    "scheme_launch_date": ""
    }

In [ ]:
# Aditya Birla Sun Life Mutual Fund
# Kotak Mahindra Mutual Fund
# Shriram Mutual fund
# PPFAS Mutual Fund
# ITI Mutual Fund
# PGIM India Mutual Fund
# BajaJ Finserv Mutual Fund
# Union Mutual Fund
# Sundaram Mutual Fund
# NJ Mutual Fund
# Samco Mutual Fund
# Bank of India Mutual Fund
# DSP Mutual Fund
# Edelweiss Mutual Fund
# Invesco Mutual Fund
# Helios Mutual Fund
# Groww Mutual Fund
# Nippon India Mutual Fund
# Quantum Mutual Fund
# Franklin Templeton Mutual Fund
# Bandhan Mutual Fund
# IL&FS Mutual Fund (IDF)
# Tata Mutual Fund
# Mirae Asset Mutual Fund
# Motilal Oswal Mutual Fund
# Old Bridge Mutual Fund
# 360 ONE Mutual Fund
# LIC Mutual Fund
# Mahindra Manulife Mutual Fund
# WhiteOak Mutual Fund
# Navi Mutual Fund
# Baroda BNP Paribas Mutual Fund
# SBI Mutual Fund
# Taurus Mutual Fund
# JM Financial Mutual Fund
# Trust Mutual Fund
# Zerodha Mutual Fund
# ICICI Prudential Mutual Fund
# Quant Mutual Fund
# Axis Mutual Fund
# Canara Robeco Mutual Fund
# HDFC Mutual Fund
# HSBC Mutual Fund
# UTI Mutual Fund
# Angel One Mutual Fund
# Unifi Mutual Fund

In [ ]:
# Axis Asset Management Company Limited
# Aditya Birla Sun Life AMC Limited
# Bank of India Investment Managers Private Limited
# Canara Robeco Asset Management Company Limited
# PGIM India Asset Management Private Limited
# DSP Investment Managers Private Limited
# Edelweiss Asset Management Limited
# Quant Money Managers Limited
# Franklin Templeton Asset Management (India) Private Limited
# HDFC Asset Management Company Limited
# HSBC Asset Management (India) Private Limited
# ICICI Prudential Asset Management Company Limited
# Bandhan AMC Limited
# 360 ONE Asset Management Limited
# IL&FS Infra Asset Management Limited
# Groww Asset Management Limited
# Invesco Asset Management (India) Private Limited
# JM Financial Asset Management Limited
# Kotak Mahindra Asset Management Company Limited(KMAMCL)
# LIC Mutual Fund Asset Management Limited
# Mahindra Manulife Investment Management Private Limited
# Mirae Asset Investment Managers (India) Private Limited
# Motilal Oswal Asset Management Company Limited
# PPFAS Asset Management Private Limited
# Quantum Asset Management Company Private Limited
# Nippon Life India Asset Management Limited
# SBI Funds Management Limited
# Shriram Asset Management Company Limited
# Sundaram Asset Management Company Limited
# Tata Asset Management Private Limited
# Taurus Asset Management Company Limited
# Union Asset Management Company Private Limited
# UTI Asset Management Company Limited
# WhiteOak Capital Asset Management Limited
# ITI Asset Management Limited
# Navi AMC Limited
# NJ Asset Management Private Limited
# Samco Asset Management Private Limited
# Trust Asset Management Private Limited
# Baroda BNP Paribas Asset Management India Private Limited
# Bajaj Finserv Asset Management Limited
# Helios Capital Asset Management (India) Private Limited
# Zerodha Asset Management Private Limited
# Old Bridge Asset Management Private Limited
# Angel One Asset Management Company Limited
# Unifi Asset Management Private Limited